In [ ]:
import requests
import json
import configparser
import datetime
import pandas as pd

#https://www.mymtaalerts.com/archive

# Initialize the configuration parser
config = configparser.ConfigParser()
config.read('config.py')

# Replace 'YOUR_API_KEY' with your actual NYCMTA service alerts API key
api_key = config['API']['api_key']

# Define the base URL for the API
base_url = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/camsys%2Fall-alerts.json'

# Set the headers with your API key
headers = {
    'x-api-key': api_key,
}

# Make a GET request to the API to fetch service alerts data
response = requests.get(base_url, headers=headers)
alerts = response.json()

In [ ]:
#establish and execute web scrape

start = []
end = []
routes = []
ltxt = []
stxt = []
atype = []

for a in range(0,len(alerts),1):
    
    try:
        
        start.append(alerts['entity'][a]['alert']['active_period'][0]['start'])
        stxt.append(alerts['entity'][a]['alert']['header_text']['translation'][0]['text'])
        ltxt.append(alerts['entity'][a]['alert']['description_text']['translation'][0]['text'].replace('\n',''))
        atype.append(alerts['entity'][a]['alert']['transit_realtime.mercury_alert']['alert_type'])
        
        try:
            
            for i in range(0,len(alerts['entity'][a]['alert']['informed_entity']),1):
                
                routes.append(alerts['entity'][a]['alert']['informed_entity'][x]['route_id'])
                
        except:
                    
                routes.append(alerts['entity'][a]['alert']['informed_entity'][0]['route_id'])
        
        try:
            
            end.append(alerts['entity'][a]['alert']['active_period'][0]['start'])
            
        except:
            
            print('Ongoing')
        
        
        print(f"{a+1}) {start[a]} | {end[a]} | {routes[a]} | {stxt[a]} | {atype[a]}")
        
    except:
        print("Scrape Failure")
        
print("-----------------------------")        
print("Scrape Completed")
print("-----------------------------")

In [ ]:
#tranfer scrape to dataframe

alerts_trsf = {
    
    'Start Time': start,
    'End Time': end,
    'Route(s)': routes,
    'Short Desc.': stxt,
    'Full Desc.': ltxt,
    'Type': atype
}

alerts_df = pd.DataFrame(data=alerts_trsf)

alerts_df

In [ ]:
#Convert unix to timestamp for final dataframe
alerts_df['Start Time_Conv'] = pd.to_datetime(alerts_df['Start Time'], unit='s').astype(str)
alerts_df['End Time_Conv'] = pd.to_datetime(alerts_df['End Time'], unit='s').astype(str)

final = alerts_df[['Start Time_Conv', 'End Time_Conv','Route(s)','Short Desc.','Full Desc.','Type']]
final